In [1]:
import pyodbc
import openpyxl
from openpyxl.styles import PatternFill, Border, Side, Alignment, Font
import pandas as pd
import numpy as np
from excelFormatter import ExcelFormatter

In [2]:
def getMaxColumnLengths(df):
    # Create a numpy array with the max length of each column in a table
    measurer = np.vectorize(len)
    result = measurer(df.values.astype(str)).max(axis=0)
    for i in range(len(result)):
        result[i] = max(result[i], len(df.columns[i]))
    return result

In [3]:
# server = 'localhost\sqlexpress' # for a named instance
# server = 'myserver,port' # to specify an alternate port
driver = "SQL Server"
server = "(local)"
database = "AdventureWorks2022"

conn = pyodbc.connect(f"Driver={driver};Server={server};Database={database};Trusted_Connection=yes;")

In [4]:
script = "SELECT AddressID, AddressLine1, City, StateProvinceID, PostalCode, ModifiedDate FROM Person.Address"

In [5]:
df = pd.read_sql_query(script, conn)
#df.astype({'PostalCode': 'int64'}, errors='ignore').dtypes

C:\Users\yipyi\AppData\Local\Temp\ipykernel_11068\2573804097.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(script, conn)


In [6]:
columnLength = getMaxColumnLengths(df)
columnLength

array([ 9, 39, 21, 15, 10, 26])

In [7]:
excelFile = "Output.xlsx"
activeSheet = "Address"
with pd.ExcelWriter(excelFile,
                    engine = "openpyxl",
                    mode = 'w') as writer:
    df.to_excel(writer, sheet_name = activeSheet, index = False)

In [8]:
wb = openpyxl.load_workbook(excelFile)
ws = wb[activeSheet]
lastCol = openpyxl.utils.get_column_letter(df.shape[1])
lastRow = df.shape[0]+1

# Format the data types
ExcelFormatter.formatTextNumber(ws, "E2:E{}".format(lastRow), "General")
ExcelFormatter.formatCellType(ws, "F2:F{}".format(lastRow), "MMM-YY")

# Set column widths to fit
ExcelFormatter.fitColumnWidth(ws, "A1:{}{}".format(lastCol, lastRow))
# Format the header row
ExcelFormatter.formatCellRange(ws, "A1:F1", Font(bold=True), PatternFill(fill_type="solid", start_color="FFFFFFFF"), 
                             Border(left=Side(border_style="thin", color="00000000"), right=Side(border_style="thin", color="00000000"), 
                                    top=Side(border_style="thin", color="00000000"), bottom=Side(border_style="thin", color="00000000")), 
                             Alignment(horizontal="center", vertical="top"))
# Format the data rows
ExcelFormatter.formatCellRange(ws, "A2:{}{}".format(lastCol, lastRow), 
                             Font(), PatternFill(fill_type="solid", start_color="FFFFFFFF"), 
                             Border(left=Side(border_style="thin", color="00000000"), right=Side(border_style="thin", color="00000000"), 
                                    top=Side(border_style="thin", color="00000000"), bottom=Side(border_style="thin", color="00000000")),
                             Alignment(horizontal="left", vertical="bottom"))
wb.save("Output1.xlsx")

In [9]:
# Format the data types
ExcelFormatter.formatTextNumber(ws, "E2:E{}".format(lastRow), "General")
ExcelFormatter.formatCellType(ws, "F2:F{}".format(lastRow), "MMM-YY")

# Set column widths
for i in range(0, len(columnLength)):
    ExcelFormatter.setColumnWidth(ws, openpyxl.utils.get_column_letter(i+1), columnLength[i])
# Format the data rows
ExcelFormatter.formatAxis(ws, ('A', lastCol), 1,
                             Font(), PatternFill(fill_type="solid", start_color="FFFFFFFF"), 
                             Border(left=Side(border_style="thin", color="00000000"), right=Side(border_style="thin", color="00000000"), 
                                    top=Side(border_style="thin", color="00000000"), bottom=Side(border_style="thin", color="00000000")),
                             Alignment(horizontal="left", vertical="bottom"))
# Format the header row
ExcelFormatter.formatAxis(ws, (1), 0, Font(bold=True), PatternFill(fill_type="solid", start_color="FFFFFFFF"), 
                             Border(left=Side(border_style="thin", color="00000000"), right=Side(border_style="thin", color="00000000"), 
                                    top=Side(border_style="thin", color="00000000"), bottom=Side(border_style="thin", color="00000000")), 
                             Alignment(horizontal="center", vertical="top"))
wb.save("Output2.xlsx")
wb.close()